In [1]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from beam_postgres.io import WriteToPostgres
import csv

# PostgreSQL connection configuration
postgres_config = {
    'host': 'localhost',
    'database': 'transport',
    'user': 'postgres',
    'password': 'postgres',
}

class ProcessCSVFn(beam.DoFn):
    def process(self, element):
        # Convert the comma-separated values to a dictionary
        data = {
            'datetime': element[0],
            'street_time': float(element[1]),
            'count': int(element[2]),
            'velocity': float(element[3])
        }
        yield data

# Create a pipeline
with beam.Pipeline(options=PipelineOptions()) as p:
    # Read CSV data
    csv_data = (
        p
        | 'Read CSV' >> beam.io.ReadFromText('output.csv', skip_header_lines=1)  # Skip the header row
        | 'Parse CSV' >> beam.Map(lambda line: next(csv.reader([line])))
    )

    # Process CSV data
    processed_data = csv_data | 'Process Data' >> beam.ParDo(ProcessCSVFn())

    # Write processed data to PostgreSQL
    processed_data | 'Write to PostgreSQL' >> WriteToPostgres(
        table='your_table_name',
        host='localhost',
        database='transport',
        user='postgres',
        password='postgres',
        batch_size=1000
    )


INFO:beam_postgres.client:Successfully execute query: INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-31 21:15:00', 1266.0, 21, 82.0952380952381);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-02-09 07:15:00', 1222.0, 23, 84.6086956521739);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-04 06:00:00', 145.0, 2, 14.5);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-02-12 12:30:00', 2188.0, 10, 33.5);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-08 03:15:00', 470.0, 2, 9.5);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-08 06:15:00', 2818.0, 65, 69.61538461538461);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-01-08 07:45:00', 10.0, 1, 23.0);
INSERT INTO your_table_name(datetime, street_time, count, velocity) VALUES('2019-02-14 16:15:00', 